<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/seirs_diferencas_sazonal_dias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import time  # Para medir o tempo de execução

# Função que simula o modelo SEIRS com beta sazonal e demografia
def simula_seirs(beta_base, beta_amplitude, nu, mu, alpha, sigma, gamma, omega, days, initial_exposed, initial_infected, initial_population):
    # Número de passos de tempo (cada passo é 1 dia)
    passos = days + 1
    tempo = np.arange(0, passos)

    # Inicializa vetores para todas as variáveis
    S = np.zeros(passos)
    E = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    N = np.zeros(passos)

    # Condições iniciais
    N[0] = initial_population
    S[0] = N[0] - initial_exposed - initial_infected
    E[0] = initial_exposed
    I[0] = initial_infected
    R[0] = 0.0

    # Iteração do modelo discreto
    for n in range(passos - 1):
        # Beta sazonal
        beta_n = beta_base * (1 + beta_amplitude * np.cos(2 * np.pi * tempo[n] / 365))

        # Atualização das variáveis com base nas equações de diferenças
        S[n+1] = S[n] + nu * N[n] - beta_n * S[n] * I[n] / N[n] - mu * S[n] + omega * R[n]
        E[n+1] = E[n] + beta_n * S[n] * I[n] / N[n] - (sigma + mu) * E[n]
        I[n+1] = I[n] + sigma * E[n] - (gamma + mu + alpha) * I[n]
        R[n+1] = R[n] + gamma * I[n] - (mu + omega) * R[n]
        N[n+1] = S[n+1] + E[n+1] + I[n+1] + R[n+1]

        # Garantir que as variáveis não sejam negativas
        S[n+1] = max(S[n+1], 0)
        E[n+1] = max(E[n+1], 0)
        I[n+1] = max(I[n+1], 0)
        R[n+1] = max(R[n+1], 0)
        N[n+1] = max(N[n+1], 0)

    return tempo, I

# Função para criar a interface interativa
def interactive_seirs():
    # Define os botões deslizantes com ajustes
    sliders = {
        "beta_base": FloatSlider(value=0.5, min=0.1, max=1.0, step=0.01, description='β base'),
        "beta_amplitude": FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='β amp'),
        "nu": FloatSlider(value=0.0001, min=0.0, max=0.001, step=0.00001, description='ν', readout_format='.5f'),
        "mu": FloatSlider(value=0.00005, min=0.0, max=0.001, step=0.00001, description='μ', readout_format='.5f'),
        "alpha": FloatSlider(value=0.001, min=0.0, max=0.01, step=0.0001, description='α', readout_format='.4f'),
        "sigma": FloatSlider(value=1/7, min=0.01, max=1.0, step=0.01, description='σ'),
        "gamma": FloatSlider(value=1/5, min=0.01, max=1.0, step=0.01, description='γ'),
        "omega": FloatSlider(value=1/365, min=0.001, max=0.1, step=0.001, description='ω', readout_format='.4f'),
        "days": IntSlider(value=1825, min=30, max=3650, step=1, description='Dias'),
        "initial_exposed": IntSlider(value=10, min=0, max=1000, step=1, description='E0'),
        "initial_infected": IntSlider(value=10, min=0, max=1000, step=1, description='I0'),
        "initial_population": IntSlider(value=100000, min=1000, max=1000000, step=1000, description='N0')
    }

    # Função para atualizar o gráfico e medir o tempo de execução
    def update_plot(**kwargs):
        # Mede o tempo de execução
        start_time = time.process_time()
        tempo, I = simula_seirs(**kwargs)
        end_time = time.process_time()

        # Calcula o tempo de CPU gasto
        cpu_time = end_time - start_time
        print(f"Tempo de CPU para simulação: {cpu_time:.4f} segundos")

        # Plota o gráfico apenas para I(t)
        plt.figure(figsize=(10, 6))
        plt.plot(tempo, I, lw=2, color='red', label='Infectados (I)')
        plt.title("Evolução de Infectados no Modelo SEIRS Sazonal")
        plt.xlabel("Dias")
        plt.ylabel("Infectados (I)")
        plt.grid()
        plt.legend()
        plt.show()

    # Cria a interface interativa
    interact(update_plot, **sliders)

# Executa a interface interativa
interactive_seirs()

interactive(children=(FloatSlider(value=0.5, description='β base', max=1.0, min=0.1, step=0.01), FloatSlider(v…